In [8]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, norm


In [9]:
# Function that create all the qasm codes and misc information about the circuits to be run
def create_all_circuits(cp):
    
    # The circuits for the experiment with input state and output distribution
    circuits = [[['X1', 'HHS', 'CZ', 'X2'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['HHS', 'Z1', 'CZ'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['HHS', 'Z1', 'Z2'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['HHS', 'Z2', 'CZ'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['Z2', 'X2'], '|00>+|11>', [0, .5, .5, 0]],
                [['X1', 'Z2'], '|0+>', [0, 0, .5, .5]],
                [['HHS', 'Z1'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['HHS', 'CZ'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['X1', 'X2'], '|00>', [0, 0, 0, 1]],
                [['HHS', 'Z2'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['X1'], '|00>+|11>', [0, .5, .5, 0]],
                [['X1'], '|0+>', [0, 0, .5, .5]],
                [['HHS'], '|00>', [0.25, 0.25, 0.25, 0.25]],
                [['Z2'], '|00>+|11>', [.5, 0, 0, .5]],
                [['Z2'], '|0+>', [.5, .5, 0, 0]],
                [['X1'], '|00>', [0, 0, 1, 0]],
                [['X2'], '|00>', [0, 1, 0, 0]],
                [[], '|00>+|11>', [.5, 0, 0, .5]],
                [[], '|0+>', [.5, .5, 0, 0]],
                [[], '|00>', [1, 0, 0, 0]]]

    # Definition of the possible gates to perform and their possible qasm implementations
    gates = ['X1','X2','Z1','Z2','HHS','CZ']

    # Count of 1- and 2-qubit physical gates
    gate_count_bare_1q = [1,1,1,1,2,2]
    gate_count_bare_2q = [0,0,0,0,0,1]

    gate_count_encoded_1q = [2,2,2,2,4,4]
    gate_count_encoded_2q = [0,0,0,0,0,0]

    # Doing the SWAP in software require swapping X1<->X2 and Z1<->Z2 depending on how many SWAPs have been done before
    indices = [[0,1,2,3,4,5],[1,0,3,2,4,5]];

    # QASM code for the gates in their bare version
    gates_qasm = [['x q['+str(cp[0])+'];\n'],
                  ['x q['+str(cp[1])+'];\n'],
                  ['z q['+str(cp[0])+'];\n'],
                  ['z q['+str(cp[1])+'];\n'],
                  ['h q['+str(cp[0])+'];\nh q['+str(cp[1])+'];\n'],
                  ['h q['+str(cp[1])+'];\ncx q['+str(cp[0])+'], q['+str(cp[1])+'];\nh q['+str(cp[1])+'];\n']]

    # QASM code for the gates in their encoded version
    gates_qasm_encoded = [['x q[1];\nx q[4];\n','x q[2];\nx q[3];\n'],
                          ['x q[1];\nx q[3];\n','x q[2];\nx q[4];\n'],
                          ['z q[1];\nz q[3];\n','z q[2];\nz q[4];\n'],
                          ['z q[1];\nz q[4];\n','z q[2];\nz q[3];\n'],
                          ['h q[1];\nh q[2];\nh q[3];\nh q[4];\n'],
                          ['s q[1];\ns q[2];\ns q[3];\ns q[4];\n']]

    #names of input states
    state_names = ['|00>','|0+>','|00>+|11>']

    # Definition of the pre- and post- circuits
    code_heading = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
"""

    bare_pre_circuit = ["","",""]
    bare_pre_circuit_count_1q = [0,1,1]
    bare_pre_circuit_count_2q = [0,0,1]
    
    bare_pre_circuit[1] = """
h q["""+str(cp[1])+"""];
barrier q["""+str(cp[0])+"""],q["""+str(cp[1])+"""];
"""
    
    bare_pre_circuit[2] = """
h q["""+str(cp[0])+"""];
cx q["""+str(cp[0])+"""], q["""+str(cp[1])+"""];
barrier q["""+str(cp[0])+"""],q["""+str(cp[1])+"""];
"""
    
    bare_post_circuit = """
measure q["""+str(cp[0])+"""] -> c["""+str(cp[0])+"""];
measure q["""+str(cp[1])+"""] -> c["""+str(cp[1])+"""];
"""
    
    encoded_pre_circuit = ["","",""]
    encoded_pre_circuit_count_1q = [11,6,2]
    encoded_pre_circuit_count_2q = [8,5,2]
    
    encoded_pre_circuit[0] = """
h q[3];
cx q[3],q[4];
cx q[4],q[2];
cx q[1],q[2];
h q[1];
h q[2];
cx q[1],q[2];
h q[1];
h q[2];
cx q[1],q[2];
cx q[3],q[2];
h q[0];
h q[1];
h q[2];
cx q[0],q[1];
cx q[0],q[2];
h q[0];
h q[1];
h q[2];
barrier q[0],q[1],q[2],q[3],q[4];
"""
    
    encoded_pre_circuit[1] = """
h q[3];
cx q[3],q[2];
cx q[1],q[2];
h q[1];
h q[2];
cx q[1],q[2];
h q[1];
h q[2];
cx q[1],q[2];
h q[4];
cx q[4],q[2];
barrier q[0],q[1],q[2],q[3],q[4];
"""
    
    encoded_pre_circuit[2] = """
h q[3];
cx q[3],q[4];
h q[1];
cx q[1],q[2];
barrier q[0],q[1],q[2],q[3],q[4];
"""
   
    encoded_post_circuit = """
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
"""
    
    #For each circuit, concatenating the state preparation code, the circuit code and the measurment code
    #Adding some misc information about the circuits on the way.
    circuit_list = []
    
    for c in circuits:
        idx = 0
        qasm_bare = code_heading + bare_pre_circuit[state_names.index(c[1])]
        qasm_encoded = code_heading + encoded_pre_circuit[state_names.index(c[1])]
        circuit_gate_count_bare_1q = bare_pre_circuit_count_1q[state_names.index(c[1])]
        circuit_gate_count_bare_2q = bare_pre_circuit_count_2q[state_names.index(c[1])]
        circuit_gate_count_encoded_1q = encoded_pre_circuit_count_1q[state_names.index(c[1])]
        circuit_gate_count_encoded_2q = encoded_pre_circuit_count_2q[state_names.index(c[1])]
        for g in c[0]:
            k = gates.index(g)
            if g=='HHS':
                idx = (idx + 1) % 2
            l = len(gates_qasm[k])
            qasm_bare += gates_qasm[indices[idx][k]][random.randrange(0,l)]
            l = len(gates_qasm_encoded[k])
            qasm_encoded += gates_qasm_encoded[k][random.randrange(0,l)]
            circuit_gate_count_bare_1q += gate_count_bare_1q[k]
            circuit_gate_count_bare_2q += gate_count_bare_2q[k]
            circuit_gate_count_encoded_1q += gate_count_encoded_1q[k]
            circuit_gate_count_encoded_2q += gate_count_encoded_2q[k]
        
        circuit_list.append({'circuit_desc':" ".join(c[0]),
                             'qasm_bare':qasm_bare + bare_post_circuit,
                             'qasm_encoded':qasm_encoded + encoded_post_circuit,
                             'nH':idx,
                             'gate_count_bare':(circuit_gate_count_bare_1q,circuit_gate_count_bare_2q),
                             'gate_count_encoded':(circuit_gate_count_encoded_1q,circuit_gate_count_encoded_2q),
                             'input_state':c[1],
                             'output_distribution':c[2]})
    return circuit_list

In [11]:
cp = [0,2]

In [12]:
create_all_circuits(cp)

[{'circuit_desc': 'X1 HHS CZ X2',
  'qasm_bare': '\nOPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg c[5];\nx q[0];\nh q[0];\nh q[2];\nh q[2];\ncx q[0], q[2];\nh q[2];\nx q[0];\n\nmeasure q[0] -> c[0];\nmeasure q[2] -> c[2];\n',
  'qasm_encoded': '\nOPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncreg c[5];\n\nh q[3];\ncx q[3],q[4];\ncx q[4],q[2];\ncx q[1],q[2];\nh q[1];\nh q[2];\ncx q[1],q[2];\nh q[1];\nh q[2];\ncx q[1],q[2];\ncx q[3],q[2];\nh q[0];\nh q[1];\nh q[2];\ncx q[0],q[1];\ncx q[0],q[2];\nh q[0];\nh q[1];\nh q[2];\nbarrier q[0],q[1],q[2],q[3],q[4];\nx q[2];\nx q[3];\nh q[1];\nh q[2];\nh q[3];\nh q[4];\ns q[1];\ns q[2];\ns q[3];\ns q[4];\nx q[1];\nx q[3];\n\nmeasure q[0] -> c[0];\nmeasure q[1] -> c[1];\nmeasure q[2] -> c[2];\nmeasure q[3] -> c[3];\nmeasure q[4] -> c[4];\n',
  'nH': 1,
  'gate_count_bare': (6, 1),
  'gate_count_encoded': (23, 8),
  'input_state': '|00>',
  'output_distribution': [0.25, 0.25, 0.25, 0.25]},
 {'circuit_desc': 'HHS Z1 CZ',
  'qasm_bare': '\